#### 상대모멘텀
- 10개의 종목을 이용하여 투자 전략 시스템

    1. 월초부터 월말까지의 수정종가를 이용하여 수익율 생성
    2. 월별 수익율이 높은 n개를 선택
    3. 해당하는 종목을 매수하여 해당하는 종목에 신호가 없어지면 매도하는 형식으로 수익율 생성

In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
import warnings

In [2]:
# warning 메시지 출력 방지
warnings.filterwarnings('ignore')

In [3]:
# 웗별 수익율을 계산하는 함수
def create_1m_rtn(_df,
                  _ticker,
                  _start = '2010-01-01',
                  _col = 'Adj Close'):
    # 복사본 생성
    result = _df.copy()
    # Date 컬럼이 존재하면 index로 변경
    if 'Date' in result.columns:
        result.set_index('Date', inplace=True)
    # index를 시계열로 변경
    result.index = pd.to_datetime(result.index)
    # 시작 시간과 기준이 되는 컬럼으로 필터링
    result = result.loc[_start:, [_col]]
    # 기준 년월 컬럼을 생성
    result['STD-YM'] = result.index.strftime('%Y-%m')
    result['1n-rtn'] = 0
    result['CODE'] = _ticker
    # 기준년월(STD-YM)의 유니크한 리스트를 생성
    ym_list = result['STD-YM'].unique()
    return result, ym_list

In [4]:
aapl = pd.read_csv("../../csv/AAPL.csv")

In [5]:
sample_aapl, ym_list = create_1m_rtn(aapl, 'AAPL')

In [6]:
sample_aapl

,Adj Close,STD-YM,1n-rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL
2010-01-06,26.402260,2010-01,0,AAPL
2010-01-07,26.353460,2010-01,0,AAPL
2010-01-08,26.528664,2010-01,0,AAPL
...,...,...,...,...
2019-06-18,198.449997,2019-06,0,AAPL
2019-06-19,197.869995,2019-06,0,AAPL
2019-06-20,199.460007,2019-06,0,AAPL


In [7]:
ym_list

array(['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06',
       '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12',
       '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
       '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
       '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
       '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
       '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
       '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
       '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
       '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
       '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06',
       '2016-07', '2016-08', '2016-09', '2016-10', 

In [8]:
# listdir() 파일의 이름의 목록
os.listdir('./data')
# glob() 파일의 경로와 이름의 목록
files = glob('./data/*.csv')

In [9]:
files

['./data/GDX.csv',
 './data/AMZN.csv',
 './data/GM.csv',
 './data/MSFT.csv',
 './data/USM.csv',
 './data/GLD.csv',
 './data/SPY.csv',
 './data/BND.csv',
 './data/AAPL.csv',
 './data/SLV.csv']

In [10]:
# 종목별 데이터프레임 생성

# 월말의 데이터를 모아주는 작업
# 월별 수익률 계산
stock_df = pd.DataFrame()
month_last_df = pd.DataFrame()

for file in files:
    # file : 주식데이터의 경로와 파일명
    # 경로와 파일명을 나눠준다.
    folder, name = os.path.split(file)
    # name : AAPL.csv --> AAPL  .csv 나눈다.
    head, tail = os.path.splitext(name)
    # head 변수는 create_1m_rtn()함수에서 _ticker 사용

    # csv 파일을 로드
    read_df = pd.read_csv(file)

    # create_1m_rtn() 함수를 호출
    price_df, ym_list = create_1m_rtn(read_df, _ticker = head)

    # 두번째 반복문 생성
    # price_df의 월별 수익율 계산하는 반복문
    for ym in ym_list:
        # ym : 기준년월
        flag = price_df['STD-YM'] == ym
        # 월초 수정 종가
        buy = price_df.loc[flag].iloc[0, 0]
        # 월말 수정 종가
        sell = price_df.loc[flag].iloc[-1, 0]
        # 수익률 계산
        m_rtn = sell / buy
        # price_df에 flag 조건과 1m_rtn 컬럼에 대입
        price_df.loc[flag, '1m_rtn'] = m_rtn
        # 월말의 데이터를 month_last_df에 단순 행 결합
        last_data = price_df.loc[flag, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat( [month_last_df, last_data] )

In [ ]:
month_last_df

,CODE,1m_rtn
Date,,
2010-01-29,GDX,0.853490
2010-02-26,GDX,1.022124
2010-03-31,GDX,0.994848
2010-04-30,GDX,1.088577
2010-05-28,GDX,0.999599
...,...,...
2019-02-28,SLV,0.980563
2019-03-29,SLV,0.998592
2019-04-30,SLV,0.990813


In [12]:
# 데이터 백업
month_rtn_df = month_last_df.copy()

In [13]:
# 인덱스를 리셋
month_rtn_df.reset_index(inplace=True)

In [14]:
# pivot_table을 이용하요 테이블을 재구조화
month_rtn_df = month_rtn_df.pivot_table(
    index = 'Date',
    columns = 'CODE',
    values = '1m_rtn'
)

In [15]:
month_rtn_df

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.897435,0.936595,1.011692,0.853490,0.965027,NaN,0.910501,0.922809,0.947586,0.870922
2010-02-26,1.050789,0.996046,1.003909,1.022124,1.009968,NaN,1.013871,0.982274,1.015404,1.004938
2010-03-31,1.124456,1.090172,0.996728,0.994848,0.995614,NaN,1.009304,1.063935,1.049976,1.117472
2010-04-30,1.106454,1.040134,1.011023,1.088577,1.046254,NaN,1.047325,1.041049,1.008574,0.996684
2010-05-28,0.964445,0.912503,1.011821,0.999599,1.027218,NaN,0.839806,0.978827,0.908766,0.968954
...,...,...,...,...,...,...,...,...,...,...
2019-02-28,1.044274,1.008363,1.001883,0.985379,0.995904,1.018051,1.094653,0.980563,1.031919,0.817688
2019-03-29,1.085615,1.065214,1.022547,1.033180,1.001067,0.947842,1.048076,0.998592,1.011783,0.974942
2019-04-30,1.049310,1.061917,1.003719,0.951275,0.997285,1.031515,1.097295,0.990813,1.028653,1.035968
